In [7]:
import pandas as pd
from datetime import datetime, timedelta
import smtplib
import ssl
import random
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

# Define constants
EXCEL_PATH = "friends_database.xlsx"
SHEET_NAME = "friend_details"

# Number of days between meeting reminders
TIER1 = 30
TIER2 = 60
TIER3 = 90


# Import excel sheet 
df = pd.read_excel(
        io = EXCEL_PATH, # Path to file
        engine = "openpyxl",
        sheet_name = SHEET_NAME, # Sheet to parse
        skiprows = None, # Indicates where to start parsing
        usecols= None,
        dtype={"mobile": "string"}) #"A:E" in case you want to use specific columns


# Convert the "date" column to date format
df["last_meeting"] = pd.to_datetime(df["last_meeting"], format="%d.%m.%Y", dayfirst=True)

# Get the current date
now = datetime.now()

# Calculate the delta between the current date and the date column in days
df["delta"] = now - df["last_meeting"]

# Show delta in days
df["delta"] = df["delta"].dt.days

# Create a new column with the corresponding threshold for each tier
df["threshold"] = df["friendship_tier"].apply(lambda x: TIER1 if x == 1 else (TIER2 if x == 2 else TIER3))

# Check whether the delta exceeds the corresponding threshold for each tier
df["exceeds_threshold"] = df["delta"] > df["threshold"]

# Get html output 
html_output = df[df["exceeds_threshold"]].to_html()

In [16]:
# Get filtered dataframe
filtered_df = df[df["exceeds_threshold"]]

# Add information to substring
bullet_list = []
target_columns = ["name", "delta", "activities", "mobile"]

# Loop over filtered dataframe
for i in range(len(filtered_df)):
    temp_name = filtered_df["name"].iloc[i]
    temp_delta = filtered_df["delta"].iloc[i]
    temp_activities = filtered_df["activities"].iloc[i]
    temp_mobile =  filtered_df["mobile"].iloc[i]

    # Random draw from activities
    temp_activity = random.choice(temp_activities.split(","))

    sample_text = f"""Hey, wie geht es dir? 🕺🏻 Hättest du lust demnächst mal wieder etwas zu unternehmen? Ggf.{temp_activity}? Gib gerne mal durch wie du Zeit hast!"""
    whatsapp_tag = f"<a href='https://wa.me/{temp_mobile}?text={sample_text}'>Send Message</a>"

    bullet_list.append(f"<li> Name: {temp_name} / Last seen before {temp_delta} days / Proposed activity: {temp_activity} / Contact: {whatsapp_tag} </li>")


In [ ]:
# Propose timeslot based on current calendar

# Easy (times from column)

# (toggle) Advanced (retrieve calendar entry for upcoming weekend and propose time-slot)

# Store activities in excel

# After confirming appointments, scan excel and transmit via google calendar API to get calendar entries

# Update table with latest appointments to overwrite "last seen"

In [17]:
# Design bullet list with the recipient
# Items to include -> Name, Days since last meeting, proposed activity, phone number

ctx = ssl.create_default_context()

# Sender, and receiver
password = "YOUR_PASSWORD"    # Your app password goes here
sender = "beckstring@gmail.com"    # Your e-mail address
receiver = "philippalexanderbecker@gmail.com" # Recipient's address

message = MIMEMultipart("alternative")

# Define subject, set sender and receiver
message["Subject"] = "Schedule Update"
message["From"] = sender
message["To"] = receiver


html = f"""<html>
  <body>
  Good Morning!
  Here is your weekly briefing whom you should meet again!
    <p>
      <h3> Meeting / Activity Proposal </h3>
        {' '.join(bullet_list)}
    </p>
  </body>
</html>
"""
message.attach(MIMEText(html, "html"))


with smtplib.SMTP_SSL("smtp.gmail.com", port=465, context=ctx) as server:
    server.login(sender, password)
    server.sendmail(sender, receiver, message.as_string())